# Run calibration 1.4.2

Here we test running the volcanic forcing time series annually with calibration v1.4.2, developed specifically for this volcanic forcing time series.

- Run all 1000 volcanic scenarios pairing each ensemble member with a volcanic forcing history = 1000
- Run three SSPs = 3
- Repeat the 1000 ensemble members with zero future volcanic forcing (another 1000 x 3)
- Run scenario, all ensemble members, zero volcanic with stochastic variability on (+ 1000 x 3)

3 x (1000 x 3) = 9000 ensemble members

Ensemble members are rebased to 1850-1900 warming.

**Update 11.03.2024**: using CMIP6 historical rather than CMIP7 v0.

In [ ]:
import os

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import pooch
import xarray as xr
from tqdm.auto import tqdm

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

In [ ]:
scenarios = ["ssp119", "ssp245", "ssp585"]

In [ ]:
df_solar = pd.read_csv(
    "../data/solar-forcing/forcing_yearly_1750-2300.csv", index_col=0
)
df_volcanic_hist = pd.read_csv(
    "../data/volcanic-forcing/yearly/forcing_yearly_cmip6_1850-2014.csv", index_col=0
)

In [ ]:
# need to introduce the volcanic scaling fron the configs at this point
df_configs = pd.read_csv(
    "../data/fair2.1-parameters/calibration-1.4.2/calibrated_constrained_parameters.csv",
    index_col=0,
)
valid_all = df_configs.index

In [ ]:
volcanic_forcing = np.zeros((352, 1000))
for volcanic_config in range(1000):  # entry zero = zero mean forcing
    df_yearly_scen = pd.read_csv(
        f"../data/volcanic-forcing/yearly/forcing_yearly_scen{volcanic_config+1}.csv", index_col=0
    )
    volcanic_forcing[:101, volcanic_config] = 0
    volcanic_forcing[101:266, volcanic_config] = df_configs['fscale_Volcanic'].values[volcanic_config] * (
        df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'] - 
        df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'].loc[1850:2014].values.mean()
    )
    volcanic_forcing[266:, volcanic_config] = df_configs['fscale_Volcanic'].values[volcanic_config] * (
        df_yearly_scen['Global yearly mean TOA ERF (W/m2)'] - 
        df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'].loc[1850:2014].values.mean()
    )

In [ ]:
pl.plot(volcanic_forcing);

In [ ]:
volcanic_forcing[101:266, :].mean()

In [ ]:
df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'].loc[1850:2014].values.mean()

In [ ]:
volcanic_forcing[266:, :].mean()

In [ ]:
df_solar

In [ ]:
df_methane = pd.read_csv(
    "../data/fair2.1-parameters/calibration-1.4.2/CH4_lifetime.csv",
    index_col=0,
)

In [ ]:
da_emissions = xr.load_dataarray(
    "../data/emissions/ssp_emissions_1750-2500.nc"
)

In [ ]:
da = da_emissions.loc[dict(config="unspecified", scenario=scenarios)][:351, ...]
fe = da.expand_dims(dim=["config"], axis=(2))

In [ ]:
#volcanic_config_names = [f'volc{i}' for i in range(1001)]

In [ ]:
output = np.zeros((252, 2, 1001))

In [ ]:
solar_forcing = df_solar["solar_erf"].loc[1750:2101].values.squeeze()

f = FAIR(ch4_method="Thornhill2021")
f.define_time(1750, 2101, 1)
f.define_scenarios(scenarios)
f.define_configs(valid_all)
species, properties = read_properties()
species.remove("Halon-1202")
species.remove("NOx aviation")
species.remove("Contrails")
f.define_species(species, properties)
f.allocate()

# solar and volcanic forcing
fill(
    f.forcing,
    volcanic_forcing[:, None],
    specie="Volcanic",
)

fill(
    f.forcing,
    solar_forcing[:, None, None] * np.array(df_configs.loc[:, "fscale_solar_amplitude"])[None, None, :],
    specie="Solar",
)

f.emissions = fe.drop_vars("config") * np.ones((1, 1, 1000, 1))


# climate response
fill(
    f.climate_configs["ocean_heat_capacity"],
    df_configs.loc[:, "clim_c1":"clim_c3"].values,
)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs.loc[:, "clim_kappa1":"clim_kappa3"].values,
)  # not massively robust, since relies on kappa1, kappa2, kappa3 being in adjacent cols
fill(
    f.climate_configs["deep_ocean_efficacy"],
    df_configs.loc[:, "clim_epsilon"],
)
fill(
    f.climate_configs["gamma_autocorrelation"],
    df_configs.loc[:, "clim_gamma"],
)
fill(f.climate_configs["sigma_eta"], df_configs.loc[:, "clim_sigma_eta"])
fill(f.climate_configs["sigma_xi"], df_configs.loc[:, "clim_sigma_xi"])
#fill(f.climate_configs["seed"], df_configs["seed"])
fill(f.climate_configs["stochastic_run"], False)
#fill(f.climate_configs["stochastic_run"], True)
#fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs.loc[:, "clim_F_4xCO2"])
    
# species level
f.fill_species_configs()
    
# carbon cycle
fill(f.species_configs["iirf_0"], df_configs.loc[:, "cc_r0"], specie="CO2")
fill(
    f.species_configs["iirf_airborne"],
    df_configs.loc[:, "cc_rA"],
    specie="CO2",
)
fill(
    f.species_configs["iirf_uptake"], df_configs.loc[:, "cc_rU"], specie="CO2"
)
fill(
    f.species_configs["iirf_temperature"],
    df_configs.loc[:, "cc_rT"],
    specie="CO2",
)
    
# aerosol indirect
fill(f.species_configs["aci_scale"], df_configs.loc[:, "aci_beta"])
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[:, "aci_shape_so2"],
    specie="Sulfur",
)
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[:, "aci_shape_bc"],
    specie="BC",
)
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[:, "aci_shape_oc"],
    specie="OC",
)
    
# methane lifetime baseline and sensitivity
fill(
    f.species_configs["unperturbed_lifetime"],
    df_methane.loc["historical_best", "base"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "CH4"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "N2O"],
    specie="N2O",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "VOC"],
    specie="VOC",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "NOx"],
    specie="NOx",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "HC"],
    specie="Equivalent effective stratospheric chlorine",
)
fill(
    f.species_configs["lifetime_temperature_sensitivity"],
    df_methane.loc["historical_best", "temp"],
)
    
# emissions adjustments for N2O and CH4
fill(f.species_configs["baseline_emissions"], 19.019783117809567, specie="CH4")
fill(f.species_configs["baseline_emissions"], 0.08602230754, specie="N2O")
fill(f.species_configs["baseline_emissions"], 19.423526730206152, specie="NOx")
    
# aerosol direct
for specie in [
    "BC",
    "CH4",
    "N2O",
    "NH3",
    "NOx",
    "OC",
    "Sulfur",
    "VOC",
    "Equivalent effective stratospheric chlorine",
]:
    fill(
        f.species_configs["erfari_radiative_efficiency"],
        df_configs.loc[:, f"ari_{specie}"],
        specie=specie,
    )
    
# forcing scaling
for specie in [
    "CO2",
    "CH4",
    "N2O",
    "Stratospheric water vapour",
    "Light absorbing particles on snow and ice",
    "Land use",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs.loc[:, f"fscale_{specie}"],
        specie=specie,
    )
    
for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs.loc[:, "fscale_minorGHG"],
        specie=specie,
    )
    
# ozone
for specie in [
    "CH4",
    "N2O",
    "Equivalent effective stratospheric chlorine",
    "CO",
    "VOC",
    "NOx",
]:
    fill(
        f.species_configs["ozone_radiative_efficiency"],
        df_configs.loc[:, f"o3_{specie}"],
        specie=specie,
    )
    
# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs["baseline_concentration"],
    df_configs.loc[:, "cc_co2_concentration_1750"],
    specie="CO2",
)
    
# initial conditions
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

f.run(progress=True)

weights = np.zeros((352, 3, 1000))
weights[100, :, :] = 0.5
weights[101:151, :, :] = 1
weights[151, :, :] = 0.5
weights = xr.DataArray(
    weights, 
    dims=f.temperature.sel(layer=0).dims, 
    coords=f.temperature.sel(layer=0).coords
)

output = f.temperature.sel(layer=0) - f.temperature.sel(layer=0).weighted(weights).mean(dim="timebounds")

In [ ]:
pl.plot(
    f.timebounds,
    (
        f.temperature.sel(layer=0) - f.temperature.sel(layer=0).weighted(weights).mean(dim="timebounds")
    ).sel(scenario='ssp119')
);

In [ ]:
(
    f.temperature.sel(layer=0) - f.temperature.sel(layer=0).weighted(weights).mean(dim="timebounds")
).sel(scenario='ssp119').median(dim='config')

In [ ]:
# SAVE OUT OUTPUT
os.makedirs('../output', exist_ok=True)

In [ ]:
ds = xr.Dataset(
    data_vars = dict(
        temperature = output,
    ),
    coords = dict(
        timebounds = np.arange(1850, 2102),
        scenario = scenarios,
        config = valid_all,
    )
)

In [ ]:
ds

In [ ]:
# intermediate dump
ds.to_netcdf('../output/stochastic_volcanoes.nc')

In [ ]:
# do stochastic run
stochastic_output = np.zeros((252, 3, 1001))

solar_forcing = df_solar["solar_erf"].loc[1750:2101].values.squeeze()

f = FAIR(ch4_method="Thornhill2021")
f.define_time(1750, 2101, 1)
f.define_scenarios(scenarios)
f.define_configs(valid_all)
species, properties = read_properties()
species.remove("Halon-1202")
species.remove("NOx aviation")
species.remove("Contrails")
f.define_species(species, properties)
f.allocate()

# solar and volcanic forcing
fill(
    f.forcing,
    volcanic_forcing[:, None],
    specie="Volcanic",
)

fill(
    f.forcing,
    solar_forcing[:, None, None] * np.array(df_configs.loc[:, "fscale_solar_amplitude"])[None, None, :],
    specie="Solar",
)

f.emissions = fe.drop_vars("config") * np.ones((1, 1, 1000, 1))


# climate response
fill(
    f.climate_configs["ocean_heat_capacity"],
    df_configs.loc[:, "clim_c1":"clim_c3"].values,
)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs.loc[:, "clim_kappa1":"clim_kappa3"].values,
)  # not massively robust, since relies on kappa1, kappa2, kappa3 being in adjacent cols
fill(
    f.climate_configs["deep_ocean_efficacy"],
    df_configs.loc[:, "clim_epsilon"],
)
fill(
    f.climate_configs["gamma_autocorrelation"],
    df_configs.loc[:, "clim_gamma"],
)
fill(f.climate_configs["sigma_eta"], df_configs.loc[:, "clim_sigma_eta"])
fill(f.climate_configs["sigma_xi"], df_configs.loc[:, "clim_sigma_xi"])
fill(f.climate_configs["seed"], df_configs["seed"])
fill(f.climate_configs["stochastic_run"], False)
fill(f.climate_configs["stochastic_run"], True)
fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs.loc[:, "clim_F_4xCO2"])
    
# species level
f.fill_species_configs()
    
# carbon cycle
fill(f.species_configs["iirf_0"], df_configs.loc[:, "cc_r0"], specie="CO2")
fill(
    f.species_configs["iirf_airborne"],
    df_configs.loc[:, "cc_rA"],
    specie="CO2",
)
fill(
    f.species_configs["iirf_uptake"], df_configs.loc[:, "cc_rU"], specie="CO2"
)
fill(
    f.species_configs["iirf_temperature"],
    df_configs.loc[:, "cc_rT"],
    specie="CO2",
)
    
# aerosol indirect
fill(f.species_configs["aci_scale"], df_configs.loc[:, "aci_beta"])
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[:, "aci_shape_so2"],
    specie="Sulfur",
)
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[:, "aci_shape_bc"],
    specie="BC",
)
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[:, "aci_shape_oc"],
    specie="OC",
)
    
# methane lifetime baseline and sensitivity
fill(
    f.species_configs["unperturbed_lifetime"],
    df_methane.loc["historical_best", "base"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "CH4"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "N2O"],
    specie="N2O",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "VOC"],
    specie="VOC",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "NOx"],
    specie="NOx",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "HC"],
    specie="Equivalent effective stratospheric chlorine",
)
fill(
    f.species_configs["lifetime_temperature_sensitivity"],
    df_methane.loc["historical_best", "temp"],
)
    
# emissions adjustments for N2O and CH4
fill(f.species_configs["baseline_emissions"], 19.019783117809567, specie="CH4")
fill(f.species_configs["baseline_emissions"], 0.08602230754, specie="N2O")
fill(f.species_configs["baseline_emissions"], 19.423526730206152, specie="NOx")
    
# aerosol direct
for specie in [
    "BC",
    "CH4",
    "N2O",
    "NH3",
    "NOx",
    "OC",
    "Sulfur",
    "VOC",
    "Equivalent effective stratospheric chlorine",
]:
    fill(
        f.species_configs["erfari_radiative_efficiency"],
        df_configs.loc[:, f"ari_{specie}"],
        specie=specie,
    )
    
# forcing scaling
for specie in [
    "CO2",
    "CH4",
    "N2O",
    "Stratospheric water vapour",
    "Light absorbing particles on snow and ice",
    "Land use",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs.loc[:, f"fscale_{specie}"],
        specie=specie,
    )
    
for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs.loc[:, "fscale_minorGHG"],
        specie=specie,
    )
    
# ozone
for specie in [
    "CH4",
    "N2O",
    "Equivalent effective stratospheric chlorine",
    "CO",
    "VOC",
    "NOx",
]:
    fill(
        f.species_configs["ozone_radiative_efficiency"],
        df_configs.loc[:, f"o3_{specie}"],
        specie=specie,
    )
    
# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs["baseline_concentration"],
    df_configs.loc[:, "cc_co2_concentration_1750"],
    specie="CO2",
)
    
# initial conditions
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

f.run(progress=True)

weights = np.zeros((352, 3, 1000))
weights[100, :, :] = 0.5
weights[101:151, :, :] = 1
weights[151, :, :] = 0.5
weights = xr.DataArray(
    weights, 
    dims=f.temperature.sel(layer=0).dims, 
    coords=f.temperature.sel(layer=0).coords
)

stochastic_output = f.temperature.sel(layer=0) - f.temperature.sel(layer=0).weighted(weights).mean(dim="timebounds")

In [ ]:
pl.plot(
    f.timebounds,
    (
        f.temperature.sel(layer=0) - f.temperature.sel(layer=0).weighted(weights).mean(dim="timebounds")
    ).sel(scenario='ssp119')
);

In [ ]:
pl.plot(stochastic_output[:, 0, :] - output[:, 0, :]);

In [ ]:
ds_sf = xr.Dataset(
    data_vars = dict(
        temperature = stochastic_output,
    ),
    coords = dict(
        timebounds = np.arange(1850, 2102),
        scenario = scenarios,
        config = valid_all,
    )
)

In [ ]:
# final dump
ds_sf.to_netcdf('../output/stochastic_volcanoes_stochastic_climate.nc')

In [ ]:
novolc_output = np.zeros((352, 3, 1000))

solar_forcing = df_solar["solar_erf"].loc[1750:2101].values.squeeze()

f = FAIR(ch4_method="Thornhill2021")
f.define_time(1750, 2101, 1)
f.define_scenarios(scenarios)
f.define_configs(valid_all)
species, properties = read_properties()
species.remove("Halon-1202")
species.remove("NOx aviation")
species.remove("Contrails")
f.define_species(species, properties)
f.allocate()

# solar and volcanic forcing
fill(
    f.forcing,
    volcanic_forcing[:266, None],
    specie="Volcanic",
    timebounds=np.arange(1750, 2016),
)
fill(
    f.forcing,
    0,
    specie="Volcanic",
    timebounds=np.arange(2016, 2102),
)

fill(
    f.forcing,
    solar_forcing[:, None, None] * np.array(df_configs.loc[:, "fscale_solar_amplitude"])[None, None, :],
    specie="Solar",
)

f.emissions = fe.drop_vars("config") * np.ones((1, 1, 1000, 1))


# climate response
fill(
    f.climate_configs["ocean_heat_capacity"],
    df_configs.loc[:, "clim_c1":"clim_c3"].values,
)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs.loc[:, "clim_kappa1":"clim_kappa3"].values,
)  # not massively robust, since relies on kappa1, kappa2, kappa3 being in adjacent cols
fill(
    f.climate_configs["deep_ocean_efficacy"],
    df_configs.loc[:, "clim_epsilon"],
)
fill(
    f.climate_configs["gamma_autocorrelation"],
    df_configs.loc[:, "clim_gamma"],
)
fill(f.climate_configs["sigma_eta"], df_configs.loc[:, "clim_sigma_eta"])
fill(f.climate_configs["sigma_xi"], df_configs.loc[:, "clim_sigma_xi"])
#fill(f.climate_configs["seed"], df_configs["seed"])
fill(f.climate_configs["stochastic_run"], False)
#fill(f.climate_configs["stochastic_run"], True)
#fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs.loc[:, "clim_F_4xCO2"])
    
# species level
f.fill_species_configs()
    
# carbon cycle
fill(f.species_configs["iirf_0"], df_configs.loc[:, "cc_r0"], specie="CO2")
fill(
    f.species_configs["iirf_airborne"],
    df_configs.loc[:, "cc_rA"],
    specie="CO2",
)
fill(
    f.species_configs["iirf_uptake"], df_configs.loc[:, "cc_rU"], specie="CO2"
)
fill(
    f.species_configs["iirf_temperature"],
    df_configs.loc[:, "cc_rT"],
    specie="CO2",
)
    
# aerosol indirect
fill(f.species_configs["aci_scale"], df_configs.loc[:, "aci_beta"])
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[:, "aci_shape_so2"],
    specie="Sulfur",
)
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[:, "aci_shape_bc"],
    specie="BC",
)
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[:, "aci_shape_oc"],
    specie="OC",
)
    
# methane lifetime baseline and sensitivity
fill(
    f.species_configs["unperturbed_lifetime"],
    df_methane.loc["historical_best", "base"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "CH4"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "N2O"],
    specie="N2O",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "VOC"],
    specie="VOC",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "NOx"],
    specie="NOx",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "HC"],
    specie="Equivalent effective stratospheric chlorine",
)
fill(
    f.species_configs["lifetime_temperature_sensitivity"],
    df_methane.loc["historical_best", "temp"],
)
    
# emissions adjustments for N2O and CH4
fill(f.species_configs["baseline_emissions"], 19.019783117809567, specie="CH4")
fill(f.species_configs["baseline_emissions"], 0.08602230754, specie="N2O")
fill(f.species_configs["baseline_emissions"], 19.423526730206152, specie="NOx")
    
# aerosol direct
for specie in [
    "BC",
    "CH4",
    "N2O",
    "NH3",
    "NOx",
    "OC",
    "Sulfur",
    "VOC",
    "Equivalent effective stratospheric chlorine",
]:
    fill(
        f.species_configs["erfari_radiative_efficiency"],
        df_configs.loc[:, f"ari_{specie}"],
        specie=specie,
    )
    
# forcing scaling
for specie in [
    "CO2",
    "CH4",
    "N2O",
    "Stratospheric water vapour",
    "Light absorbing particles on snow and ice",
    "Land use",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs.loc[:, f"fscale_{specie}"],
        specie=specie,
    )
    
for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs.loc[:, "fscale_minorGHG"],
        specie=specie,
    )
    
# ozone
for specie in [
    "CH4",
    "N2O",
    "Equivalent effective stratospheric chlorine",
    "CO",
    "VOC",
    "NOx",
]:
    fill(
        f.species_configs["ozone_radiative_efficiency"],
        df_configs.loc[:, f"o3_{specie}"],
        specie=specie,
    )
    
# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs["baseline_concentration"],
    df_configs.loc[:, "cc_co2_concentration_1750"],
    specie="CO2",
)
    
# initial conditions
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

f.run(progress=True)

weights = np.zeros((352, 3, 1000))
weights[100, :, :] = 0.5
weights[101:151, :, :] = 1
weights[151, :, :] = 0.5
weights = xr.DataArray(
    weights, 
    dims=f.temperature.sel(layer=0).dims, 
    coords=f.temperature.sel(layer=0).coords
)

novolc_output = f.temperature.sel(layer=0) - f.temperature.sel(layer=0).weighted(weights).mean(dim="timebounds")

In [ ]:
pl.plot(novolc_output[:, 0, :] - output[:, 0, :]);

In [ ]:
(
    f.temperature.sel(layer=0) - f.temperature.sel(layer=0).weighted(weights).mean(dim="timebounds")
).sel(scenario='ssp119').median(dim='config')

In [ ]:
ds_nv = xr.Dataset(
    data_vars = dict(
        temperature = novolc_output,
    ),
    coords = dict(
        timebounds = np.arange(1850, 2102),
        scenario = scenarios,
        config = valid_all,
    )
)

In [ ]:
ds_nv.to_netcdf('../output/no_future_volcanoes.nc')